
#IMPORTAÇÃO DE DEPENDÊNCIAS





In [74]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import NearestNeighbors
import numpy as np
from sklearn.metrics import accuracy_score
import sklearn.metrics.pairwise as pw
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
!pip install fuzzywuzzy
from fuzzywuzzy import process
from fuzzywuzzy import fuzz


##Filmes

In [22]:
url = 'https://drive.google.com/file/d/11soEDwLvKtI6dR3-3OkUWyXnHqU-_0bP/view?usp=sharing'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
df_filme = pd.read_csv(url,sep=',')


##Notas

In [23]:
url = 'https://drive.google.com/file/d/1nrzhbmeK5OTEoqiCQ5lsayk8_q3FLnQc/view?usp=sharing'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
df_nota = pd.read_csv(url,sep=',')

##Dados

In [24]:
url = 'https://drive.google.com/file/d/1TayoFh9h-1Ghtk0Tm_LQWjqPyLlyJGzQ/view?usp=sharing'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
df_dados = pd.read_csv(url,sep=',')

##Tags

In [25]:
url = 'https://drive.google.com/file/d/1utKL4qufR0OmACaAeNubINH5n-Vkb4U2/view?usp=sharing'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
df_tag = pd.read_csv(url,sep=',')

In [ ]:
#filmes = pd.read_csv("/content/drive/MyDrive/IIC/IIC Final/Dados.csv", sep="\t")
#nota = pd.read_csv("/content/drive/MyDrive/IIC/IIC Final/star.csv", sep="\t")


In [26]:
df_filme.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


In [7]:
df_nota.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [27]:
df_dados.head()

,Name,Year,Stars,Score,Time,Votes,Total,Tags,Directors_Cast,Discription
0,Pulp Fiction,1994,8.9,94.0,154,"1,871,051",$107.93,\nCrime Drama,Quentin Tarantino John Travolta Uma Thurman Sa...,\nThe lives of two mob hitmen a boxer a gangst...
1,The Amazing Spider-Man 2,2014,6.6,53.0,142,"425,529",$202.85,\nAction Adventure Fantasy,Marc Webb Andrew Garfield Emma Stone Jamie Fox...,\nWhen New York is put under siege by Oscorp i...
2,The Shawshank Redemption,1994,9.3,80.0,142,"2,409,436",$28.34,\nDrama,Frank Darabont Tim Robbins Morgan Freeman Bob ...,\nTwo imprisoned men bond over a number of yea...
3,Star Wars: Episode IV - A New Hope,1977,8.6,90.0,121,"1,255,464",$322.74,\nAction Adventure Fantasy,George Lucas Mark Hamill Harrison Ford Carrie ...,\nLuke Skywalker joins forces with a Jedi Knig...
4,Back to the Future,1985,8.5,87.0,116,"1,087,878",$210.61,\nAdventure Comedy Sci-Fi,Robert Zemeckis Michael J. Fox Christopher Llo...,\nMarty McFly a 17-year-old high school studen...


In [28]:
df_tag.head()

,movieId,tag
0,1,"Owned,imdb top 250,Pixar,Pixar,time travel,chi..."
1,2,"Robin Williams,time travel,fantasy,based on ch..."
2,3,"funny,best friend,duringcreditsstinger,fishing..."
3,4,"based on novel or book,chick flick,divorce,int..."
4,5,"aging,baby,confidence,contraception,daughter,g..."


In [31]:
print("Colunas em df_nota:", df_nota.columns)

Colunas em df_nota: Index(['userId', 'movieId', 'rating'], dtype='object')


In [29]:
df_filme.shape

(9737, 4)

In [30]:
df_nota.shape

(100836, 3)

#LIMPEZA DE DADOS

In [13]:
duplicados = df_filme[df_filme.duplicated(keep='first')]
print(duplicados)

Empty DataFrame
Columns: [movieId, title, genres, year]
Index: []


In [14]:
df_filme_set = set(df_filme['movieId'])
df_nota_set = set(df_nota['movieId'])

In [15]:
df_nota.drop_duplicates(subset ='movieId', keep='first', inplace=True)

#COMBINANDO DATAFRAMES

In [32]:
df_juntar = df_filme.merge(df_nota, on='movieId')
df_juntar.head()

,movieId,title,genres,year,userId,rating
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995,1,4.0
1,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995,5,4.0
2,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995,7,4.5
3,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995,15,2.5
4,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995,17,4.5


In [35]:
filmes = pd.pivot_table(df_juntar, index='title', columns='userId', values='rating').fillna(0)
filmes.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
'71,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
'Hellboy': The Seeds of Creation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Round Midnight,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Salem's Lot,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Til There Was You,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#REMOÇÃO DE VALORES NULOS

In [36]:
rec = pw.cosine_similarity(filmes)
rec

array([[1.        , 0.        , 0.        , ..., 0.32732684, 0.        ,
        0.        ],
       [0.        , 1.        , 0.70710678, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.70710678, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.32732684, 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [37]:
rec_df = pd.DataFrame(rec, columns=filmes.index, index=filmes.index)
rec_df.head()

title,'71,'Hellboy': The Seeds of Creation,'Round Midnight,'Salem's Lot,'Til There Was You,'Tis the Season for Love,"'burbs, The",'night Mother,(500) Days of Summer,*batteries not included,...,Zulu,[REC],[REC]²,[REC]³ 3 Génesis,anohana: The Flower We Saw That Day - The Movie,eXistenZ,xXx,xXx: State of the Union,¡Three Amigos!,À nous la liberté (Freedom for Us)
title,,,,,,,,,,,,,,,,,,,,,
'71,1.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.141653,0.0,...,0.0,0.342055,0.543305,0.707107,0.0,0.0,0.139431,0.327327,0.0,0.0
'Hellboy': The Seeds of Creation,0.0,1.000000,0.707107,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
'Round Midnight,0.0,0.707107,1.000000,0.000000,0.000000,0.0,0.176777,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
'Salem's Lot,0.0,0.000000,0.000000,1.000000,0.857493,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
'Til There Was You,0.0,0.000000,0.000000,0.857493,1.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0


#Resultado obtido utilizando o Colaborative Filtering

In [41]:
cossine_df = pd.DataFrame(rec_df['Finding Nemo'].sort_values(ascending=False))
cossine_df.columns = ['Recomendações']
cossine_df.head(20)

,Recomendações
title,
Finding Nemo,1.000000
"Incredibles, The",0.726374
Shrek,0.701435
"Monsters, Inc.",0.697340
Pirates of the Caribbean: The Curse of the Black Pearl,0.652265
Shrek 2,0.625074
Catch Me If You Can,0.594174
"Lord of the Rings: The Return of the King, The",0.587321
"Lord of the Rings: The Two Towers, The",0.568078


In [42]:
df_dados.head()

,Name,Year,Stars,Score,Time,Votes,Total,Tags,Directors_Cast,Discription
0,Pulp Fiction,1994,8.9,94.0,154,"1,871,051",$107.93,\nCrime Drama,Quentin Tarantino John Travolta Uma Thurman Sa...,\nThe lives of two mob hitmen a boxer a gangst...
1,The Amazing Spider-Man 2,2014,6.6,53.0,142,"425,529",$202.85,\nAction Adventure Fantasy,Marc Webb Andrew Garfield Emma Stone Jamie Fox...,\nWhen New York is put under siege by Oscorp i...
2,The Shawshank Redemption,1994,9.3,80.0,142,"2,409,436",$28.34,\nDrama,Frank Darabont Tim Robbins Morgan Freeman Bob ...,\nTwo imprisoned men bond over a number of yea...
3,Star Wars: Episode IV - A New Hope,1977,8.6,90.0,121,"1,255,464",$322.74,\nAction Adventure Fantasy,George Lucas Mark Hamill Harrison Ford Carrie ...,\nLuke Skywalker joins forces with a Jedi Knig...
4,Back to the Future,1985,8.5,87.0,116,"1,087,878",$210.61,\nAdventure Comedy Sci-Fi,Robert Zemeckis Michael J. Fox Christopher Llo...,\nMarty McFly a 17-year-old high school studen...


In [43]:
df_tag.head()

,movieId,tag
0,1,"Owned,imdb top 250,Pixar,Pixar,time travel,chi..."
1,2,"Robin Williams,time travel,fantasy,based on ch..."
2,3,"funny,best friend,duringcreditsstinger,fishing..."
3,4,"based on novel or book,chick flick,divorce,int..."
4,5,"aging,baby,confidence,contraception,daughter,g..."


#transformando movieId em inteiro

In [47]:
df_filme['movieId'] = df_filme['movieId'].apply(lambda x: str(x))

In [48]:
df_dados.shape

(9937, 10)

In [49]:
df_tag.shape

(45256, 2)

In [51]:
df2 = df_filme.merge(df_dados, left_on='title', right_on='Name', how='left')
df2 = df2.merge(df_tag, left_on='movieId', right_on='movieId', how='left')
df2['Infos'] = df2['genres'] + str(df2['Directors_Cast']) + str(df2['Discription']) + df2['tag']
df2.head()

,movieId,title,genres,year,Name,Year,Stars,Score,Time,Votes,Total,Tags,Directors_Cast,Discription,tag,Infos
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995,Toy Story,1995,8.3,95.0,81,"908,794",$191.80,\nAnimation Adventure Comedy,John Lasseter Tom Hanks Tim Allen Don Rickles ...,\nA cowboy doll is profoundly threatened and j...,"Owned,imdb top 250,Pixar,Pixar,time travel,chi...",Adventure|Animation|Children|Comedy|Fantasy0 ...
1,2,Jumanji,Adventure|Children|Fantasy,1995,Jumanji,1995,7.0,39.0,104,"316,485",$100.48,\nAdventure Comedy Family,Joe Johnston Robin Williams Kirsten Dunst Bonn...,\nWhen two kids find and play a magical board ...,"Robin Williams,time travel,fantasy,based on ch...",Adventure|Children|Fantasy0 John Lasset...
2,3,Grumpier Old Men,Comedy|Romance,1995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"funny,best friend,duringcreditsstinger,fishing...",Comedy|Romance0 John Lasseter Tom Hanks...
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995,Waiting to Exhale,1995,6.0,NaN,124,"9,911",$67.05,\nComedy Drama Romance,Forest Whitaker Whitney Houston Angela Bassett...,NaN,"based on novel or book,chick flick,divorce,int...",Comedy|Drama|Romance0 John Lasseter Tom...
4,5,Father of the Bride Part II,Comedy,1995,Father of the Bride Part II,1995,6.1,49.0,106,"35,472",$76.59,\nComedy Family Romance,Charles Shyer Steve Martin Diane Keaton Martin...,\nGeorge Banks must deal not only with the pre...,"aging,baby,confidence,contraception,daughter,g...",Comedy0 John Lasseter Tom Hanks Tim All...


In [53]:
vetor = TfidfVectorizer()
Tfidf = vetor.fit_transform(df2['Infos'].apply(lambda x: np.str_(x)))

In [54]:
similaridade = cosine_similarity(Tfidf)
similaridade

array([[1.        , 0.08465854, 0.01312877, ..., 0.01361825, 0.02186222,
        0.01361825],
       [0.08465854, 1.        , 0.02747468, ..., 0.0599022 , 0.0949832 ,
        0.0599022 ],
       [0.01312877, 0.02747468, 1.        , ..., 0.19453706, 0.21472584,
        0.19453706],
       ...,
       [0.01361825, 0.0599022 , 0.19453706, ..., 1.        , 0.68030435,
        1.        ],
       [0.02186222, 0.0949832 , 0.21472584, ..., 0.68030435, 1.        ,
        0.68030435],
       [0.01361825, 0.0599022 , 0.19453706, ..., 1.        , 0.68030435,
        1.        ]])

In [56]:
similaridade_df = pd.DataFrame(similaridade, columns=df2['title'], index=df2['title'])
similaridade_df.head ()

title,Toy Story,Jumanji,Grumpier Old Men,Waiting to Exhale,Father of the Bride Part II,Heat,Heat,Sabrina,Tom and Huck,Sudden Death,...,Gintama: The Movie,anohana: The Flower We Saw That Day - The Movie,Silver Spoon,Love Live! The School Idol Movie,Jon Stewart Has Left the Building,Black Butler: Book of the Atlantic,No Game No Life: Zero,Flint,Bungo Stray Dogs: Dead Apple,Andrew Dice Clay: Dice Rules
title,,,,,,,,,,,,,,,,,,,,,
Toy Story,1.000000,0.084659,0.013129,0.016002,0.033845,0.007786,0.007786,0.006416,0.052864,0.007285,...,0.099869,0.013618,0.013618,0.013618,0.013618,0.013618,0.013618,0.013618,0.021862,0.013618
Jumanji,0.084659,1.000000,0.027475,0.151885,0.103882,0.003845,0.003845,0.052921,0.087719,0.041849,...,0.076600,0.059902,0.059902,0.059902,0.059902,0.059902,0.059902,0.059902,0.094983,0.059902
Grumpier Old Men,0.013129,0.027475,1.000000,0.139910,0.196165,0.046368,0.046368,0.043952,0.131067,0.072365,...,0.179095,0.194537,0.194537,0.194537,0.194537,0.194537,0.194537,0.194537,0.214726,0.194537
Waiting to Exhale,0.016002,0.151885,0.139910,1.000000,0.278759,0.019550,0.019550,0.255034,0.314388,0.151062,...,0.373854,0.407665,0.407665,0.407665,0.407665,0.407665,0.407665,0.407665,0.448173,0.407665
Father of the Bride Part II,0.033845,0.103882,0.196165,0.278759,1.000000,0.011208,0.011208,0.081509,0.261189,0.144008,...,0.356444,0.372773,0.372773,0.372773,0.372773,0.372773,0.372773,0.372773,0.427902,0.372773


#Resultado obtido utilizando o Content-Based-Filtering

In [59]:
resultado_df = pd.DataFrame(similaridade_df['Finding Nemo'].sort_values(ascending=False))
resultado_df.columns = ['Recomendações']
resultado_df.head(30)


,Recomendações
title,
Finding Nemo,1.000000
"Bug's Life, A",0.776843
Finding Dory,0.762913
Toy Story,0.745477
Toy Story 2,0.730738
"Monsters, Inc.",0.713622
Ice Age,0.636715
"Incredibles, The",0.636557
For the Birds,0.621745


#KNN

In [60]:
df_dados.head()

,Name,Year,Stars,Score,Time,Votes,Total,Tags,Directors_Cast,Discription
0,Pulp Fiction,1994,8.9,94.0,154,"1,871,051",$107.93,\nCrime Drama,Quentin Tarantino John Travolta Uma Thurman Sa...,\nThe lives of two mob hitmen a boxer a gangst...
1,The Amazing Spider-Man 2,2014,6.6,53.0,142,"425,529",$202.85,\nAction Adventure Fantasy,Marc Webb Andrew Garfield Emma Stone Jamie Fox...,\nWhen New York is put under siege by Oscorp i...
2,The Shawshank Redemption,1994,9.3,80.0,142,"2,409,436",$28.34,\nDrama,Frank Darabont Tim Robbins Morgan Freeman Bob ...,\nTwo imprisoned men bond over a number of yea...
3,Star Wars: Episode IV - A New Hope,1977,8.6,90.0,121,"1,255,464",$322.74,\nAction Adventure Fantasy,George Lucas Mark Hamill Harrison Ford Carrie ...,\nLuke Skywalker joins forces with a Jedi Knig...
4,Back to the Future,1985,8.5,87.0,116,"1,087,878",$210.61,\nAdventure Comedy Sci-Fi,Robert Zemeckis Michael J. Fox Christopher Llo...,\nMarty McFly a 17-year-old high school studen...


In [61]:
df_tag.head()

,movieId,tag
0,1,"Owned,imdb top 250,Pixar,Pixar,time travel,chi..."
1,2,"Robin Williams,time travel,fantasy,based on ch..."
2,3,"funny,best friend,duringcreditsstinger,fishing..."
3,4,"based on novel or book,chick flick,divorce,int..."
4,5,"aging,baby,confidence,contraception,daughter,g..."


In [63]:
filme_df2 = df_filme.merge(df_dados, left_on='title', right_on='Name', how='left')
filme_df2 = filme_df2.merge(df_tag, left_on='movieId', right_on='movieId', how='left')
filme_df2['Infos'] = filme_df2['genres'] + str(filme_df2['Directors_Cast']) + str(filme_df2['Discription']) + filme_df2['tag']
filme_df2.head()

,movieId,title,genres,year,Name,Year,Stars,Score,Time,Votes,Total,Tags,Directors_Cast,Discription,tag,Infos
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995,Toy Story,1995,8.3,95.0,81,"908,794",$191.80,\nAnimation Adventure Comedy,John Lasseter Tom Hanks Tim Allen Don Rickles ...,\nA cowboy doll is profoundly threatened and j...,"Owned,imdb top 250,Pixar,Pixar,time travel,chi...",Adventure|Animation|Children|Comedy|Fantasy0 ...
1,2,Jumanji,Adventure|Children|Fantasy,1995,Jumanji,1995,7.0,39.0,104,"316,485",$100.48,\nAdventure Comedy Family,Joe Johnston Robin Williams Kirsten Dunst Bonn...,\nWhen two kids find and play a magical board ...,"Robin Williams,time travel,fantasy,based on ch...",Adventure|Children|Fantasy0 John Lasset...
2,3,Grumpier Old Men,Comedy|Romance,1995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"funny,best friend,duringcreditsstinger,fishing...",Comedy|Romance0 John Lasseter Tom Hanks...
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995,Waiting to Exhale,1995,6.0,NaN,124,"9,911",$67.05,\nComedy Drama Romance,Forest Whitaker Whitney Houston Angela Bassett...,NaN,"based on novel or book,chick flick,divorce,int...",Comedy|Drama|Romance0 John Lasseter Tom...
4,5,Father of the Bride Part II,Comedy,1995,Father of the Bride Part II,1995,6.1,49.0,106,"35,472",$76.59,\nComedy Family Romance,Charles Shyer Steve Martin Diane Keaton Martin...,\nGeorge Banks must deal not only with the pre...,"aging,baby,confidence,contraception,daughter,g...",Comedy0 John Lasseter Tom Hanks Tim All...


In [72]:
filme_df2['Infos'].fillna('', inplace=True)

In [77]:
vectorizer = CountVectorizer()
info_matrix = vectorizer.fit_transform(filme_df2['Infos'])

In [78]:
knnsim = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=30)
knnsim.fit(info_matrix)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=30)

In [79]:
def get_recommendations(title, knn_model, vectorizer, dataframe, matrix):
    # Encontrar o índice do filme no DataFrame
    idx = dataframe.index[dataframe['Finding Nemo'] == title].tolist()[0]

    # Obter os vizinhos mais próximos
    distances, indices = knn_model.kneighbors(matrix[idx])

    # Exibir os filmes recomendados
    print(f"Recomendações para '{title}':")
    for i, index in enumerate(indices.flatten()):
        if i == 0:
            continue  # O próprio filme está na lista de recomendações, ignorando-o
        print(f"{i}: {dataframe['Finding Nemo'][index]}, Distância: {distances.flatten()[i]:.4f}")

In [83]:
from tabulate import tabulate

def recommender(movie_title, knn_model, vectorizer, dataframe, matrix, num_recommendations=30):

    idx = dataframe.index[dataframe['title'] == movie_title].tolist()[0]


    distances, indices = knn_model.kneighbors(matrix[idx])


    recommended_movies = []
    for i, index in enumerate(indices.flatten()):
        if i == 0:
            continue
        recommended_movies.append((dataframe['title'][index], distances.flatten()[i]))


    recommended_movies = sorted(recommended_movies, key=lambda x: x[1])

    # Criar uma lista de listas para a tabela
    table_data = [[i+1, title, f"{distance:.4f}"] for i, (title, distance) in enumerate(recommended_movies[:num_recommendations])]

    # Cabeçalho da tabela
    headers = ["Rank", "Movie Title", "Distance"]

    # Retorna a tabela formatada
    return tabulate(table_data, headers=headers, tablefmt="grid")


movie_title = 'Finding Nemo'
table_result = recommender(movie_title, knnsim, vectorizer, filme_df2, info_matrix)


print(table_result)


+--------+-------------------------------------------------+------------+
|   Rank | Movie Title                                     |   Distance |
+========+=================================================+============+
|      1 | Bug's Life, A                                   |     0.2451 |
+--------+-------------------------------------------------+------------+
|      2 | Toy Story                                       |     0.2799 |
+--------+-------------------------------------------------+------------+
|      3 | Monsters, Inc.                                  |     0.3008 |
+--------+-------------------------------------------------+------------+
|      4 | Ice Age                                         |     0.3114 |
+--------+-------------------------------------------------+------------+
|      5 | Toy Story 2                                     |     0.316  |
+--------+-------------------------------------------------+------------+
|      6 | Finding Dory               